In [2]:
!pip install selenium

# 필요한 패키지 불러오기

In [3]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pickle

# 호텔별 URL을 가져오기

In [6]:
# url 함수
def make_url(Url):
    import time
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome()
    driver.get(Url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    click1 = driver.find_element(By.XPATH,'//*[@id="__next"]/div/header/section/div/div[1]/section[2]/div/div/div/div/div[2]/div/div[1]/span[1]/span')
    click1.click()
    time.sleep(6)
    click2 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[1]')
    click2.click()
    click3 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[2]')
    click3.click()
    time.sleep(2)
    click4 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/footer/div/button[2]')
    click4.click()
    time.sleep(2)
    #무한 스크롤
    
    time.sleep(1)
    
    SCROLL_PAUSE_TIME = 1.5
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height  

    soup = BeautifulSoup(html, 'html.parser')
    data = soup.select("section > div > div > a")
        
    for d in data:
        hotelurl.append(d.attrs['href'])
    # driver.close()
  

In [8]:
hotelurl = []
make_url("https://www.yanolja.com/hotel/r-900587?pageKey=1724955969696")

# 리뷰 수집

In [16]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

hotel = []
star = []
reviews = []
rev_sum_pos = []
rev_sum_neg = []
columns = ['hotel','star' ,'reviews', 'Review Summary Pos', 'Review Summary Neg','url' ]   


for url in hotelurl:
    cnt = 0
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(1)
    Url = "https://www.yanolja.com"+url
    driver.get(Url)
    WebDriverWait(driver, 10)
    # try:
    # # 요소 찾기 시도
    try : 
        rev_sum_pos.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text)
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except NoSuchElementException:
        rev_sum_pos.append("")
    try:   
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except TimeoutException:
        pass
    try :    
        rev_sum_neg.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text) 
    except NoSuchElementException:
        rev_sum_neg.append("") 
    # except NoSuchElementException:
    # # 요소가 없을 경우 공백 문자열 설정
    #     rev_sum_pos = ''
    #     rev_sum_neg = ''
    
    hotel.append(driver.find_element(By.CSS_SELECTOR,'#__next > div > div > main > article > div.css-1cc3d9 > div > div.css-1e56347 > div.css-11vo59c > h1').text)
    
    # 리뷰 페이지 클릭 (Click review page)
    try:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-xa55ms'))
        )
    except NoSuchElementException :
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    except TimeoutException:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    element2.click()
    # 무한 스크롤
    import time
    time.sleep(1)
    
    # SCROLL_PAUSE_TIME = 1.5
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "css-stl2cn")))
    element.click()
    
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//div[text()='평점 낮은순']")))
    element.click()
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    max = int(driver.find_element(By.CLASS_NAME,"css-12rhouj").text.replace("(","").replace(")",""))
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if max > 1000:
            if(cnt>20):
                break
            cnt +=1
        if new_height == last_height:
            break
    
        last_height = new_height
    
    # 스크롤 끝나고 페이지 elements 가져오기
    star.append(float(driver.find_element(By.CLASS_NAME, 'css-u41ekg').text))
    soup = bs(driver.page_source, "lxml")
    
    # 특정 클래스 이름을 가진 모든 요소 찾기
    elements = soup.find_all('p')
    temp = ""
    # 데이터 추출 
    review = driver.find_elements(By.CLASS_NAME,"css-c92dc4")
    for i in review:
        temp += i.text
    reviews.append(temp)     
    
df = pd.DataFrame({"Hotel" : hotel, "Star" : star ,"Reviews" : reviews, "Review Pos" : rev_sum_pos,"Review Neg" : rev_sum_neg,"Url": hotelurl})   
df["Url"] ="https://www.yanolja.com"+df["Url"]

In [18]:
with open('강원.pickle', 'wb') as f:
    pickle.dump(df, f)

In [20]:
df

,Hotel,Star,Reviews,Review Pos,Review Neg,Url
0,UH FLAT The 속초,4.7,청결에 신경써주시면 좋을것 같습니다최악은 화장실 정화조냄새.변기냄새트랩을 안했는지 ...,"최근에 개장하여 깨끗하고 현대적인 시설을 갖추고 있으며, 직원들이 친절하다는 평이 ...","신축된 숙소로, 청결한 관리 상태와 좋은 위치가 장점입니다. 중앙시장이 가까워 접근...",https://www.yanolja.com/hotel/10055957
1,강릉 경포 호텔 마레,4.8,가격 대비 별로네요. 침대가 너무 딱딱하고 소음도 잘들려요208호 였는데 오션뷰가 ...,"경포해변 바로 앞에 위치해 있어 해수욕과 수영에 편리하며, 주변에 다양한 먹거리가 ...","해변과 가까운 위치에 있는 숙소로, 전반적으로 협소한 공간과 청결도에 대한 의견이 ...",https://www.yanolja.com/hotel/10046839
2,디그니티 호텔,4.6,View가 졸라아쉬움 ㅠㅠ이 호텔에서 숙박하려면 목숨걸고 예약하셔야겠습니다.\n\n...,넓고 깨끗한 객실과 친절한 직원들이 특징입니다. 낙산해수욕장과 가까워 가족이나 연인...,"가족 단위 여행객에게 적합하며, 넓고 깨끗한 객실을 제공합니다. 그러나 주변 음식점...",https://www.yanolja.com/hotel/3015116
3,속초 아이파크스위트 호텔 앤 레지던스,4.7,"숙소 깔끔하고 오션뷰도 좋은점은 좋은거같습니다\n다만 이용도 못하는 인덕션,세탁기는...","깨끗한 객실과 훌륭한 바다 전망으로 호평받고 있으며, 인피니티풀과 조식도 높은 평가...","숙소는 전반적으로 깔끔하고 오션뷰와 같은 훌륭한 경관을 제공하나, 주요 문제로 엘리...",https://www.yanolja.com/hotel/10051581
4,센텀마크 호텔 양양,4.7,모든면이 부족하였어요~잘못예약으로 예약후 1시간뒤 취소하려하였으나 예약 하루전인데도...,"청결하고 쾌적한 환경을 제공하며, 고객 서비스의 친절함이 돋보이는 곳입니다. 객실은...","저렴한 가격대에 위치해 있으며, 전반적으로 청결하고 깔끔한 환경을 제공합니다. 주변...",https://www.yanolja.com/hotel/1000114137
5,더베네치아스위트 호텔,4.3,여지껏 다닌 호텔 중에 제일 최악\n리뷰보면 개선한다는 댓글은 잘 달음 그걸 믿고 ...,"청결도와 가성비가 뛰어난 레지던스형으로, 각종 시설이 잘 갖춰져 있어 편리한 이용이...","청결도와 주차 관련 불만이 주를 이룹니다. 주차 공간이 협소하고, 기계식 주차로 인...",https://www.yanolja.com/hotel/3016731
6,어반스테이 속초등대해변,4.6,호텔을 예약했는데요 레지던스였네요\n속았네요 사기당한기분에 돌아왔어요\n예약시 ( ...,"해변과 가까운 숙소로, 뷰가 뛰어나고 청결 상태가 우수하며, 다양한 시설이 갖춰져 ...",해안가 근처에 위치한 이 숙소는 전반적으로 깨끗한 내부와 다양한 편의시설을 갖추고 ...,https://www.yanolja.com/hotel/10043064
7,어반스테이 속초해변 C,4.6,호텔인줄 알았으나 말그대로 에어비앤비 같은 스테이하우스.\n스위트룸 와서 쓰레기 뒷...,"해수욕장과 가까운 위치에 있는 숙소로, 객실은 넓고 깔끔하며 편의 시설이 잘 갖춰져...","해변과 가까운 이상적인 위치에 있으며, 주변에는 다양한 편의시설과 관광명소가 있어 ...",https://www.yanolja.com/hotel/10041421
8,강릉 파인아트라벨,4.3,사진에 있는게 객실 들어가자마자 짐만 내려놓고 발견한 겁니다. 바닥에 누구의 것인지...,"해변과 가까운 위치에 있는 숙소로, 오션뷰를 제공하며 내부 인테리어는 깔끔하고 아늑...","숙소는 바다 전망과 접근성이 좋은 위치에 있으며, 주차 공간이 넉넉하다는 장점이 있...",https://www.yanolja.com/hotel/1000114001
9,홈 마리나 속초 - Part of Banyan Group,4.5,성수기에 비싼 요금내고 간거였는데 전체적으로 별로네요 특히 인피니티풀 넘 더워서 놀...,전반적으로 청결하고 친절한 서비스로 높은 만족도를 자랑합니다. 침구와 룸은 깨끗하며...,"성수기에 비해 전반적인 만족도가 낮다는 후기가 많습니다. 특히, 인피니티 풀의 높은...",https://www.yanolja.com/hotel/10058240


In [1]:
len(df)

NameError: name 'df' is not defined